# Feature Selection of LSTM 

# Import libraries

In [1]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

import matplotlib.pyplot as plt
import pandas as pd;
import numpy as np;
import sklearn
import time

# empty list to append metric values
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []
time_count = []

# Import keras libraries, packages and data:

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# import data
data_full = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

data_full = data_full.loc[data_full.index > 2018070000, :]

# reset index
data_full.reset_index(inplace = True)
data_full.drop('index', axis = 1, inplace = True)

# fill nan values
data_full.fillna(method = 'ffill', inplace = True)

# parameters
features_num = 15
steps = 96
n_hidden = 1
units = 150
batch_size = 96
epochs = 180

# Functions for LSTM:

In [3]:
# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# design the LSTM
def regressor_tunning(features_num = features_num, bias_initializer = initializers.Ones() , kernel_initializer = 'he_normal'):
    model = Sequential()
    if n_hidden == 0:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.RMSprop(lr = 0.0001)
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
    return model

# First prediction with one feature:

In [ ]:
start_time = time.time()

# get data ready with one feature
data = data_full.loc[:,['PrevDay', 'Offers']]

# divide data into train and test 
data_train, data_test = train_test_split(
             data, test_size = 0.15, shuffle=False) 

sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# divide features and labels
X_train = data_train[:, 0] 
y_train = data_train[:, -1]
X_test = data_test[:, 0] 
y_test = data_test[:, -1]

# divide data into train and test 
X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)

# function to split data into correct shape for RNN for 1 feature only
def split_data_1(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# put data into correct shape
X_train, y_train = split_data_1(X_train, y_train, steps)
X_test, y_test = split_data_1(X_test, y_test, steps)
X_val, y_val = split_data_1(X_val, y_val, steps)

X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

features_num = 1

model = regressor_tunning(features_num = features_num)

# fitting the LSTM to the training set
history = model.fit(X_train,
                    y_train, 
                    batch_size = batch_size, 
                    epochs = epochs,
                    shuffle = False, 
                    validation_data = (X_val, y_val))

# make new predicitons with test set
y_pred = model.predict(X_test, batch_size = batch_size)
    
# prices col = 15 (inverso should not be used as scalling was made with the whole data set)
y_pred = (y_pred * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])
y_test = (y_test * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])

# smal adjustment
y_test = pd.Series(y_test)
y_test.replace(0, 0.0001,inplace = True)
 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mae_error = mae(y_test, y_pred)
  
rmse_gen.append(rmse_error)
mae_gen.append(mae_error)
    
# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================
    
y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mae_spike = mae(y_test_spike, y_pred_spike)
    
rmse_spi.append(rmse_spike)
mae_spi.append(mae_spike)
    
# =============================================================================
# Metric evaluation on normal regions
# =============================================================================
    
# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)
 
# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
  
# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mae_normal = mae(y_test_normal, y_pred_normal)
    
rmse_nor.append(rmse_normal)
mae_nor.append(mae_normal)
    
elapsed_time = time.time() - start_time

time_count.append(elapsed_time)

Epoch 1/180
65/65 [==============================] - 34s 516ms/step - loss: 0.1580 - mse: 0.1580 - mae: 0.3146 - val_loss: 0.0046 - val_mse: 0.0046 - val_mae: 0.0609
Epoch 2/180
65/65 [==============================] - 32s 500ms/step - loss: 0.1074 - mse: 0.1074 - mae: 0.2614 - val_loss: 0.0031 - val_mse: 0.0031 - val_mae: 0.0452
Epoch 3/180
65/65 [==============================] - 30s 460ms/step - loss: 0.0680 - mse: 0.0680 - mae: 0.2076 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0364
Epoch 4/180
65/65 [==============================] - 25s 382ms/step - loss: 0.0496 - mse: 0.0496 - mae: 0.1773 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0290
Epoch 5/180
65/65 [==============================] - 33s 512ms/step - loss: 0.0324 - mse: 0.0324 - mae: 0.1421 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0250
Epoch 6/180
65/65 [==============================] - 33s 503ms/step - loss: 0.0224 - mse: 0.0224 - mae: 0.1188 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0316
Epoc

65/65 [==============================] - 26s 405ms/step - loss: 9.4180e-04 - mse: 9.4180e-04 - mae: 0.0182 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0194
Epoch 50/180
65/65 [==============================] - 31s 479ms/step - loss: 9.4066e-04 - mse: 9.4066e-04 - mae: 0.0181 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 51/180
65/65 [==============================] - 34s 522ms/step - loss: 9.3970e-04 - mse: 9.3970e-04 - mae: 0.0181 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 52/180
65/65 [==============================] - 36s 553ms/step - loss: 9.4099e-04 - mse: 9.4099e-04 - mae: 0.0181 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0192
Epoch 53/180
53/65 [=======================>......] - ETA: 6s - loss: 8.5406e-04 - mse: 8.5406e-04 - mae: 0.0179

# Loop to continue testing other features:

In [ ]:
# features list; order made according to Linear Regression FS
features_list = ['APXP', 
                 'LOLP',  
                 'In_gen',
                 'Ren_R',
                 'DA_imb_France', 
                 'Rene',
                 'ratio_offers_vol',
                 'DA_price_france',
                 'TSDF',
                 'dino_bin',
                 'DA_margin',
                 'Im_Pr']

best_score = rmse_error

# LOOP STARTS
for i in features_list:
    start_time = time.time()
    
    # data recovery for in case there is no improvement
    data_recovery = data
    
    # update feature number
    features_num = features_num + 1
    
    # add new feature
    data = pd.concat([data, data_full.loc[:,i]], axis = 1)    

    # divide data into train and test 
    data_train, data_test = train_test_split(
             data, test_size = 0.15, shuffle=False)    
    
    from sklearn.preprocessing import MinMaxScaler

    # data scaling  (including offer (y))
    sc_X = MinMaxScaler()
    data_train = sc_X.fit_transform(data_train)
    data_test = sc_X.transform(data_test)
    
    # divide features and labels
    X_train = data_train[:, 0: features_num] 
    y_train = data_train[:, -1]
    X_test = data_test[:, 0:features_num] 
    y_test = data_test[:, -1] 

    # divide data into train and test 
    X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)

    # put data into correct shape
    X_train, y_train = split_data(X_train, y_train, steps)
    X_test, y_test = split_data(X_test, y_test, steps)
    X_val, y_val = split_data(X_val, y_val, steps)

    X_train = cut_data(X_train, batch_size)
    y_train = cut_data(y_train, batch_size)
    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)
    X_val = cut_data(X_val, batch_size)
    y_val = cut_data(y_val, batch_size)

    model = regressor_tunning(features_num = features_num)
    
    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = epochs,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15 (inverso should not be used as scalling was made with the whole data set)
    y_pred = (y_pred * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])
    y_test = (y_test * sc_X.data_range_[-1]) + (sc_X.data_min_[-1])

    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)
    
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)
    
    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)
    
    # condition of improvement for FS
    if best_score < rmse_gen[-1]:
        data = data_recovery
        features_num = features_num - 1
    else:
        data = data
        best_score = rmse_error

In [ ]:
X_test.shape

In [ ]:
results = pd.DataFrame({                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
    
                        'time': time_count})

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
results.min()

In [ ]:
!pip install matplotlib

import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

dates_labels = ['12 ',
                '10 ',
                '8 ',
                '6 ',
                '4 ',
                '2 ']

plt.figure(figsize=(10,4))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('LSTM: Averaged RMSE for different\n predictive windows')
plt.plot(rmse_gen, label = 'Overall error')
plt.plot(rmse_spi, label = 'Spike regions')
plt.plot(rmse_nor, label = 'Normal regions')
plt.legend()
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Predictive window (in months)')
plt.xticks([0,1,2,3,4,5], dates_labels)
plt.tight_layout()
plt.savefig('RMSE_predictive_window.png')

plt.figure(figsize=(10,4))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('LSTM: Averaged MAE for different\n predictive windows')
plt.plot(mae_gen, label = 'Overall error')
plt.plot(mae_spi, label = 'Spike regions')
plt.plot(mae_nor, label = 'Normal regions')
plt.legend()
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Predictive window (in months)')
plt.xticks([0,1,2,3,4,5], dates_labels)
plt.tight_layout()
plt.savefig('MAE_predictive_window.png')